# Algorithm Implementation

by [Lei You](http://user.it.uu.se/~leiyo378)

In this document, we solve the flexible TTI allocation problem to optimum, by using our proposed algorithm. The formulation of the original problem is as below.

$$
\begin{align}
\max_{\mathbf{x}} \quad & \sum_{b\in\mathcal{B}}\sum_{k\in\mathcal{K}^{(c)}} r_{b,k}x_{b,k} \\
s.t. \quad & \sum_{b\in\mathcal{B}}r_{b,k}x_{b,k}\geq q_{k} \quad k\in\mathcal{K}^{(\ell)} \\
           & \sum_{b\in\mathcal{B}}\sum_{k\in\mathcal{K}}a_{b,i}x_{b,k}\leq 1 \quad i\in\mathcal{I} \\
           & x_{b,k}\in\{0,1\}\quad b\in\mathcal{B},~k\in\mathcal{K}
\end{align}
$$

We have $x_{b,k}=1$ if and only if physical resource block (PRB) $b$ is allocated to service $k$.

Recall that $\tau_k$ is the maximum tolerant latency of service $k$, and $d_b$ is the end time of the PRB $b$. The constraints for the latency is imposed, by letting $r_{b,k}$ follow the rule:
$$
r_{b,k}=\left\{
\begin{array}{ll}
0 & \text{if } \tau_k-d_b<0 \\
\text{capacity} & \text{otherwise}
\end{array}
\right.
$$

Here we go.

In [31]:
import sys
import scipy.io
import numpy
import math
import csv
from operator import add

from gurobipy import *

# set the directory path
import os
folder_name = os.getcwd()

epsilon = 10e-6

import time

The sets $\mathcal{B}$, $\mathcal{K}^{(\ell)}$, $\mathcal{K}^{(c)}$, $\mathcal{K}$, and $\mathcal{I}$ are read by the following code.

In [32]:
# # set of physical layer blocks (PRBs)
with open('B.csv', 'rb') as f:
    B_csv = csv.reader(f)
    B = list(B_csv)
    B = [item for sublist in B for item in sublist] # flatten list
    B = map(int, map(float, B)) # convert to int
    
# # set of latency services    
with open('Kl.csv', 'rb') as f:
    Kl_csv = csv.reader(f)
    Kl = list(Kl_csv)
    Kl = [item for sublist in Kl for item in sublist] # flatten list
    Kl = map(int, map(float, Kl)) # convert to int
    
# # set of capacity services    
with open('Kc.csv', 'rb') as f:
    Kc_csv = csv.reader(f)
    Kc = list(Kc_csv)
    Kc = [item for sublist in Kc for item in sublist] # flatten list
    Kc = map(int, map(float, Kc)) # convert to int
    
# # set of all services
K = Kl + Kc

# # set of resource units (RUs)
with open('I.csv', 'rb') as f:
    I_csv = csv.reader(f)
    I = list(I_csv)
    I = [item for sublist in I for item in sublist] # flatten list
    I = map(int, map(float, I)) # convert to int

The parameters $\mathbf{r}$, $\mathbf{q}$, and $\mathbf{a}$ are read below.

In [33]:
# # matrix r
with open('r.csv', 'rb') as f:
    r_csv = csv.reader(f)
    r = list(r_csv)
    r = [ map(int,map(float,x)) for x in r] # convert to int

# # vector q, only for Kl
with open('q.csv', 'rb') as f:
    q_csv = csv.reader(f)
    q = list(q_csv)
    q = [item for sublist in q for item in sublist] # flatten list
    q = map(int, map(float, q)) # convert to int
    
# # matrix a
with open('a.csv', 'rb') as f:
    a_csv = csv.reader(f)
    a = list(a_csv)
    a = [ map(int,map(float,x)) for x in a] # convert to int

The following code reads the overlapping relationship between any two PRBs.

In [34]:
# if conflict_PRBs[b1][b2] == True, then the PRBs b1 b2 cannot be used simultaneously
with open('conflict_PRB.csv', 'rb') as f:
    conflict_PRB_csv = csv.reader(f)
    conflict_PRB = list(conflict_PRB_csv)
    conflict_PRB = [ map(lambda y: int(float(y))==1, x) for x in conflict_PRB]

The original problem is relaxed by Lagrangian with $\mathbf{\lambda}>\mathbf{0}$:

The Lagrangian is as follows.

$$
\begin{align}
g(\mathbf{\lambda})=\max_{\mathbf{x}} \quad & \sum_{b\in\mathcal{B}}\sum_{k\in\mathcal{K}^{(c)}} r_{b,k}x_{b,k} + \sum_{i\in\mathcal{I}}\lambda_i(1-\sum_{b\in\mathcal{B}}\sum_{k\in\mathcal{K}}a_{b,i}x_{b,k})\\
s.t. \quad & \sum_{b\in\mathcal{B}}r_{b,k}x_{b,k}\geq q_{k} \quad k\in\mathcal{K}^{(\ell)} \\
           & x_{b,k}\in\{0,1\}\quad b\in\mathcal{B},~k\in\mathcal{K}
\end{align}
$$

For any fixed $\mathbf{\lambda}$, the orginal problem decomposes to two problems in respect of $\mathcal{K}^{(c)}$ and $\mathcal{K}^{(\ell)}$. For the sake of presentation, we denote 
$$\alpha_b = \sum_{i\in\mathcal{I}}\lambda_i a_{b,i}$$ 

In [35]:
def getAlpha(lam):
    alpha=[]
    for b in B:
        alpha_b = sum( lam[i]*a[b][i] for i in I ) 
        alpha.append(alpha_b)
    return alpha

For $\mathcal{K}^{(c)}$, the problem is as follows.
$$
\begin{align}
\max_{\mathbf{x}} \quad & \sum_{b\in\mathcal{B}}\sum_{k\in\mathcal{K}^{(c)}} r_{b,k}x_{b,k} - \sum_{b\in\mathcal{B}}\sum_{k\in\mathcal{K}^{(c)}}\alpha_{b}x_{b,k}\\
s.t. \quad & x_{b,k}\in\{0,1\}\quad b\in\mathcal{B},~k\in\mathcal{K}^{(c)}
\end{align}
$$

In the above formulation, it might happen that one PRB is allocated to multiple services simultaneously, which definitely leads to PRB overlap. Therefore we add an extra constraint such that the problem becomes:
$$
\begin{align}
\max_{\mathbf{x}} \quad & \sum_{b\in\mathcal{B}}\sum_{k\in\mathcal{K}^{(c)}}x_{b,k}( r_{b,k} - \alpha_{b})\\
% s.t. \quad & \sum_{k\in\mathcal{K}^{(c)}}x_{b,k}\leq 1 \quad b\in\mathcal{B} \\
           & x_{b,k}\in\{0,1\}\quad b\in\mathcal{B},~k\in\mathcal{K}^{(c)}
\end{align}
$$

It can be optimally solved by:
For each $b\in\mathcal{B}$, we select one $k$, such that.
$
r_{b,k}-\alpha_b > 0 \text{ and } k = \arg\max_{k\in\mathcal{K}^{(c)}} r_{b,k}-\alpha_b
$.
The corresponding code is as follows.

In [36]:
# The argument lam is lambda
# The function returns matrix x, with only columns in Kc being computed.
# The columns in Kl of the returned matrix x are zero vectors.
def solveP2( lam ):
    PRB_alloc = [ -1 for b in B ] # PRB_alloc[b] is the index of service that PRB b should be allocated to
                                  # PRB_alloc[b]=-1 means that PRB b is not allocated
        
    sol_x = [ [ 0 for k in K ] for b in B ] # variables to be returned
    
    alpha = getAlpha(lam)
    for b in B:
        tmp_list = [ r[b][k]-alpha[b] for k in Kc ]
        if max(tmp_list) > 0:   # PRB is allocated only if r[b][k]-alpha[b] is positive
            PRB_alloc[b] = len(Kl) + numpy.argmax(tmp_list) # Kl is added such that the value of PRB_alloc[b] is 
                                                       # coherent with the corresponding indexed position in K
    
    # Convert PRB_alloc to matrix x
    for b in B:
        k = PRB_alloc[b] # indexed service
        if k >= 0: # indicating that PRB_alloc[b] != -1
            sol_x[b][k] = 1
    
    return sol_x

For $\mathcal{K}^{(\ell)}$, we have the problem below.
$$
\begin{align}
\min_{\mathbf{x}} \quad &  \sum_{b\in\mathcal{B}}\sum_{k\in\mathcal{K}^{(\ell)}}\alpha_{b}x_{b,k}\\
s.t. \quad & \sum_{b\in\mathcal{B}}r_{b,k}x_{b,k}\geq q_{k} \quad k\in\mathcal{K}^{(\ell)} \\
           & x_{b,k}\in\{0,1\}\quad b\in\mathcal{B},~k\in\mathcal{K}^{(\ell)}
\end{align}
$$

The problem can decomposed to $|\mathcal{K}^{(\ell)}|$ knapsack problems and be optimally solved by dynamic programming. 

For $k\in\mathcal{K}^{(\ell)}$:
$$
\begin{align}
\min_{\mathbf{x}} \quad &  \sum_{b\in\mathcal{B}}\alpha_{b}x_{b,k}\\
s.t. \quad & \sum_{b\in\mathcal{B}}r_{b,k}x_{b,k}\geq q_{k}  \\
           & x_{b,k}\in\{0,1\}\quad b\in\mathcal{B}
\end{align}
$$

Though the multiple knapsack problem can still be exactly solved by dynamic programming, here we use gurobi integer programming solver instead, without loss of optimality.

In [37]:
# The argument lam is lambda
# The function returns matrix x, with only columns in Kl being computed.
# The columns in Kc of the returned matrix x are zero vectors.
def solveP3( lam ):
    # create optimization model
    modelKl = Model('Integer Programming - Kl')
    modelKl.modelSense = GRB.MINIMIZE
    modelKl.setParam('OutputFlag', False) # slience output
    
    # create varialbes for modelKl:
    xKl = []
    for b in B:
        xKl_b = []
        for k in Kl:
            xKl_b.append(modelKl.addVar(vtype=GRB.BINARY))
        xKl.append(xKl_b)
    modelKl.update()
    
    # add constraints 
    for k in Kl:
        modelKl.addConstr( sum(r[b][k]*xKl[b][k] for b in B) >= q[k] )
    modelKl.update()

    # set objective function
    alpha = getAlpha(lam)
    modelKl.setObjective(
        sum( alpha[b]*xKl[b][k] for k in Kl for b in B )
    )
    
    # solve modelKl
    modelKl.optimize()
    
    # construct variables to be returned 
    sol_x = [ [ 0 for k in K ] for b in B ]
    for b in B:
        for k in Kl:
            sol_x[b][k] = int(xKl[b][k].x)
            
    return sol_x

The solutions obtained by $\texttt{solveP2}$ and $\texttt{solveP3}$ can be merged to obtain the value of Lagrangian Dual function, by:

Next, we use subgradient descent method to solve the Lagrangian Dual problem, i.e., $\min_{\mathbf{\lambda}\geq\mathbf{0}} g(\mathbf{\lambda})$. After the completion of the gradient descent method, the obtained solution is not guaranteed to be feasible to the primal problem. The heuristic method to obtain a feasible solution is as follows: All PRBs allocation for $\mathcal{K}^{(\ell)}$ is kept so as to guarantee the latency constraints being satisfied. Then we solve $\mathcal{K}^{(c)}$ under the current lambda and fixed $\mathcal{K}^{(\ell)}$ solutions.

To achieve this goal, we need to implement a new function that solves Kc with some PRB allocations being fixed:

In [38]:
def assignBlockLD(lam, x_count):
    priority = [ [i[0] for i in sorted(enumerate(x_count[k]), key=lambda y:y[1],reverse=True)] for k in K ]
    
    best = -1
    sol_x_best = []
    for i in range(400):
        sol_x = [ [ 0 for k in K ] for b in B ] # variables to be returned
        collision = [ False for b in B ]

        Kl_rand = numpy.random.permutation(Kl)
        Kc_rand = numpy.random.permutation(Kc)
        B_rand = numpy.random.permutation(B)

        for k in Kl_rand:
            service_bit = sum(r[b][k]*sol_x[b][k] for b in B)
            while service_bit < q[k] and collision.count(False)>0: 
                for pos in range(len(priority[k])):
                    b = priority[k][pos]
                    if collision[b] == False: 
                        sol_x[b][k] = 1
                        service_bit += r[b][k]*sol_x[b][k]
                        for p in B: # set all PRB overlapping with b to be in collision
                            if conflict_PRB[b][p] == True:
                                collision[p] = True
                        break

        alpha = getAlpha(lam)
        for b in B_rand:
            alloc = True
            if collision[b]==True: # b wouldn't be allocated if in collision
                alloc = False 
            if alloc == True: 
                tmp_list = [ r[b][k]-alpha[b] for k in Kc ]
                user_to_alloc = len(Kl) + numpy.argmax(tmp_list) 
                sol_x[b][user_to_alloc] = 1
                for p in B: # set all PRB overlapping with b to be in collision
                    if conflict_PRB[b][p] == True:
                        collision[p] = True
                        
        curr = sum(r[b][k]*sol_x[b][k] for k in Kc for b in B)
        if curr > best:
            best = curr
            sol_x_best = sol_x
                    
    return sol_x_best

In [39]:
def assignBlockLP(x_count):
    sol_x = [ [ 0 for k in K ] for b in B ]
    collision = [ False for b in B ]
    priority = [ [i[0] for i in sorted(enumerate(x_count[k]), key=lambda y:y[1],reverse=True)] for k in K ]
    best_sol = [];
    
    best = -1
    for k in range(400):
        Kl_rand = numpy.random.permutation(Kl)
        Kc_rand = numpy.random.permutation(Kc)
        B_rand = numpy.random.permutation(B)

        for k in Kl_rand:
            service_bit = sum(r[b][k]*sol_x[b][k] for b in B)
            while service_bit < q[k] and collision.count(False)>0: 
                for pos in range(len(priority[k])):
                    b = priority[k][pos]
                    if collision[b] == False: 
                        sol_x[b][k] = 1
                        service_bit += r[b][k]*sol_x[b][k]
                        for p in B: # set all PRB overlapping with b to be in collision
                            if conflict_PRB[b][p] == True:
                                collision[p] = True
                        break
                
        for k in Kc_rand:
            for b in B_rand:
                if collision[b] == False and x_count[k][b] >= 0.5:
                    sol_x[b][k]=1
                    for p in B:
                        if conflict_PRB[b][p] == True:
                            collision[p] = True

        while collision.count(False)>0:
            for k in Kc_rand:
                for pos in range(len(priority[k])):
                    b = priority[k][pos]
                    if collision[b] == False:
                        sol_x[b][k] = 1
                        for p in B:
                            if conflict_PRB[b][p] == True:
                                collision[p] = True
                        break

        obj = sum( r[b][k]*int(sol_x[b][k]) for k in Kc for b in B )
        if obj > best:
            best = obj
            best_sol = sol_x

    return best_sol

The following function checks whether a solution is primal feasible.

In [40]:
def isFeasible(x):
    if (numpy.dot( numpy.dot(numpy.matrix(a).transpose(), 
                             numpy.matrix(x)), numpy.ones(len(K)) ) 
        > numpy.ones(len(I))).tolist()[0].count(True)>0:
        return False
    if (numpy.dot(numpy.multiply(numpy.matrix(r),numpy.matrix(x))[:,0:len(Kl)].transpose(),
                  numpy.ones(len(B)))<numpy.matrix(q)).tolist()[0].count(True)>0:
        return False
    return True

The gradient descent along with the heuristic afterwards is implemented as follows.

In [41]:
def LPBased(rho):
    model = Model('Linear Programming')
    model.modelSense = GRB.MAXIMIZE
    model.setParam('OutputFlag', False) # slience output

    x = []
    for b in B:
        x_b = []
        for k in K:
            x_b.append(model.addVar(vtype=GRB.CONTINUOUS, lb=0, ub=1))
        x.append(x_b)

    model.update()

    for k in Kl:
        model.addConstr( sum(r[b][k]*x[b][k] for b in B) >= q[k] )
    model.update()

    for i in I:
        model.addConstr( sum(a[b][i]*x[b][k] for k in K for b in B ) <= 1 )
    model.update()

    model.setObjective(
        sum( r[b][k]*x[b][k] for k in Kc for b in B )
    )

    model.optimize()
    
    x_count = [ [ x[b][k].x for b in B] for k in K ]
    for b in B:
        for k in K:
            if x[b][k].x < rho:
                x_count[k][b] = 0
    return x_count

In [42]:
def LDBased(M, varphi, beta, gamma):
    start_time = time.time()
    x_count_LP = LPBased(0)
    x_LP = assignBlockLP(x_count_LP)
    obj_LP =  sum(r[b][k]*x_count_LP[k][b] for k in Kc for b in B)
    
    lam = [1 for i in I]
    x_prev = [ [0 for k in K ] for b in B ]
    penalty = [ 1 for i in I ]
    x_count = [ [ 0 for b in B ] for k in K ]
    best_dual_sofar = 1e10
    eta = 0.95
    no_improve_count = 0
    
    for j in range(1, M+1): 
        
        xKc = solveP2(lam)
        xKl = solveP3(lam)
        x = (numpy.matrix(xKl) + numpy.matrix(xKc)).tolist() 

        # obtain the corresponding dual function value under current lambda

        dual = sum(r[b][k]*x[b][k] for k in Kc for b in B) + sum( lam[i]*(1-sum(a[b][i]*x[b][k] for k in K for b in B)) for i in I)
        if dual < best_dual_sofar + 0.005:
            no_improve_count = 0
            best_dual_sofar = dual
            gamma = min(gamma*1.05, 1.9)
        else:
            no_improve_count += 1

        if no_improve_count >=1:
            gamma = 0.95*gamma
            no_improve_count = 0

        numurator = abs(dual - obj_LP*beta)
        denominator = numpy.linalg.norm(penalty, 2)**2

        if j<varphi:
            mu = 10/float(j**0.5)
        else:
            mu = gamma*numurator/denominator

        penalty = (numpy.ones(len(I))- numpy.dot( numpy.dot(numpy.matrix(a).transpose(), numpy.matrix(x)), numpy.ones(len(K)))).tolist()[0]
        lam = map(lambda y:max(y,0), (numpy.matrix(lam) - mu*numpy.matrix(penalty)).tolist()[0])

        x_count = (numpy.matrix(x_count) + numpy.matrix(x).transpose()).tolist()    
        
        print 'iteration', j, 'step length=', mu, 'dual=', best_dual_sofar, 'gamma=', gamma
        
    print 'LD upper bound = ', best_dual_sofar
    print 'Solving LD costs', time.time()-start_time, 'seconds'
    return [lam, x_count]

In [43]:
x_count_LP = LPBased(0)
print 'LP upper bound =', sum(r[b][k]*x_count_LP[k][b] for k in Kc for b in B)
print 'calculating final solution (LP)... '
x_LP = assignBlockLP(x_count_LP)
print 'feasibility=', isFeasible(x_LP), '\n obj=', sum(r[b][k]*x_LP[b][k] for k in Kc for b in B)

LP upper bound = 1995.95452024
calculating final solution (LP)... 
feasibility= False 
 obj= 0


In [44]:
# lam = [1 for i in I]
result = LDBased(M=200, varphi=5, beta=0.85, gamma=0.5)
lam = result[0]
x_count_LD = result[1]
print 'calculating final solution ... '
x_LD = assignBlockLD(lam,x_count_LD)
print 'feasibility=', isFeasible(x_LD), '\n obj=', sum(r[b][k]*x_LD[b][k] for k in Kc for b in B)

iteration 1 step length= 10.0 dual= 65924 gamma= 0.525
iteration 2 step length= 7.07106781187 dual= 9075.0 gamma= 0.55125
iteration 3 step length= 5.7735026919 dual= 7754.80194847 gamma= 0.5788125
iteration 4 step length= 5.0 dual= 7130.23431034 gamma= 0.607753125
iteration 5 step length= 16.0854454939 dual= 6637.08138014 gamma= 0.63814078125
iteration 6 step length= 13.7239149557 dual= 6637.08138014 gamma= 0.606233742188
iteration 7 step length= 7.61965744485 dual= 6637.08138014 gamma= 0.575922055078
iteration 8 step length= 2.7234718694 dual= 6173.24358355 gamma= 0.604718157832
iteration 9 step length= 5.81039647432 dual= 5814.46302138 gamma= 0.634954065724
iteration 10 step length= 10.1602645547 dual= 5445.501568 gamma= 0.66670176901
iteration 11 step length= 9.27265175338 dual= 5140.50780043 gamma= 0.70003685746
iteration 12 step length= 7.73588229294 dual= 4917.04480867 gamma= 0.735038700333
iteration 13 step length= 5.89059286765 dual= 4718.98046566 gamma= 0.77179063535
iteration

iteration 102 step length= 0.862977843735 dual= 2265.37486205 gamma= 0.218407863312
iteration 103 step length= 0.569302152133 dual= 2265.37486205 gamma= 0.207487470147
iteration 104 step length= 0.418542888127 dual= 2261.37621315 gamma= 0.217861843654
iteration 105 step length= 0.478381653752 dual= 2231.91898558 gamma= 0.228754935837
iteration 106 step length= 1.02905641739 dual= 2227.81397804 gamma= 0.240192682628
iteration 107 step length= 0.754754402617 dual= 2227.81397804 gamma= 0.228183048497
iteration 108 step length= 0.326564744009 dual= 2227.81397804 gamma= 0.216773896072
iteration 109 step length= 0.562784812881 dual= 2227.81397804 gamma= 0.205935201269
iteration 110 step length= 0.829560673295 dual= 2227.81397804 gamma= 0.195638441205
iteration 111 step length= 0.946314807256 dual= 2227.81397804 gamma= 0.185856519145
iteration 112 step length= 0.446986225967 dual= 2227.81397804 gamma= 0.176563693188
iteration 113 step length= 0.269898723018 dual= 2227.81397804 gamma= 0.167735

iteration 200 step length= 0.114080310288 dual= 2134.26678043 gamma= 0.0389537688863
LD upper bound =  2134.26678043
Solving LD costs 232.890416145 seconds
calculating final solution ... 
feasibility= True 
 obj= 1677


In [45]:
x_LD = assignBlockLD(lam,x_count_LD)
print 'feasibility=', isFeasible(x_LD), '\n obj=', sum(r[b][k]*x_LD[b][k] for k in Kc for b in B)

feasibility= True 
 obj= 1677
